# Tagger comparison

I am going to compare the performance of three Russian POS-taggers - pymorphy2, Mystem and NLTK - using data from MorphoRuEval that I have left from some other lab.

First, let's load the taggers and prepare the dataset.

In [1]:
import pymorphy2
from pymystem3 import Mystem
import nltk

In [2]:
f ="unamb_sent_14_6.conllu"

In [3]:
from nltk.corpus.reader.conll import *
reader = ConllCorpusReader(root = '', fileids = [f], columntypes = ['words','pos', 'tree', 'chunk', 'ne', 'ignore'])

In [4]:
#this is needed to see what the original text looked like
import string
def assemble_string(tokens):
    s=''
    for i in range(len(tokens)):
        if i==0:
            word=tokens[i][0].upper()+tokens[i][1:]
            s = s+word
        if i>0:
            if tokens[i][0] in '(«':
                s=s+' '+tokens[i]
            elif tokens[i][0] in string.punctuation+'»':
                s = s+tokens[i]
            elif s[-1] in '(«':
                s = s + tokens[i]
            else:
                s = s + ' ' + tokens[i]
    return s

I randomly select 32 sentences from the corpus.

In [24]:
#words with original tags
words=[]
#just words
text=[]
for i in reader.iob_sents()[512:544]:
    for j in i:
        words.append((j[1],j[2]))
        text.append(j[1])

Now I have 389 words to test the taggers on.

In [130]:
len(text)

389

I'm going to need to adjust the tags of my pos-taggers to the tags of the dataset, which are mostly UD-like tags.

In [10]:
sent_list=list(reader.iob_sents())
tags = list(set([word[2] for sent in sent_list for word in sent ]))

In [51]:
tags

['ADV',
 'DET',
 'ADP',
 'NUM',
 'CONJ',
 'PART',
 'PROPN',
 'X',
 'VERB',
 'NOUN',
 'PRON',
 'PUNCT',
 'INTJ',
 'ADJ']

After looking at my taggers' tagsets (Mystem tagset: https://tech.yandex.ru/mystem/doc/grammemes-values-docpage/, pymorphy2 tagset: https://github.com/kmike/pymorphy2/blob/master/pymorphy2/tagset.py), I wrote down some rules for converting their tags to the tags of my dataset. For NLTK, I will use build-in method to convert it's tags to UD tags, so there won't be a lot of things to change.

In [198]:
nlt={'PRT':'PART'}

myst={'A':'ADJ', 'ADVPRO':'ADV', 'PR':'PREP', 'S':'NOUN', 'SPRO':'PRON', 'V':'VERB', 'APRO':'DET', 'ANUM':'ADJ', 'COM':'X'}

pym={'ADJF':'ADJ', 'ADJS':'ADJ', 'COMP':'ADJ', 'INFN':'VERB', 'PRTF':'VERB', 'PRTS':'VERB','GRND':'VERB','NUMR':'NUM', 'ADVB':'ADV', 'NPRO':'PRON', 'PRED':'ADV', 'PREP':'ADP','PRCL':'PART', 'UNKN':'X', 'PNCT':'PUNCT'}

In [25]:
article=assemble_string(text)

This is what the test data text look like.

In [26]:
article

'«Опускались сумерки. Я задал ей по-немецки несколько вопросов, на которые она отвечала невнятными восклицаниями. Сам господин Жильяр сделал из этого вывод, что больной было ранее сказано, что к ней приедет великая княгиня, и «узнавание» было основано на этом факте. Александра Теглева и великая княгиня Ольга Старая императрица была непреклонна: Она появляется в Мариинской больнице в октябре 1925 года. Ратлеф вспоминает также сцену перед отъездом и запомнившиеся ей слова: Перед отъездом она беседовала с датским послом: Мои племянницы совершенно не говорили по-немецки. Французский они освоили позже, по-немецки в семье не говорили вовсе. (…) В 1925 году ей исполнилось бы 24. Мне показалось, что госпожа Андерсон выглядит намного старше. Великая княгиня вспоминала, что разговаривать с больной было трудно. Больная не проявила интереса к этим фотографиям. Интересно также привести объяснение, которая великая княгиня Ольга Александровна дала столь убедительным для многих «великокняжеским воспом

In [7]:
morph = pymorphy2.MorphAnalyzer()

In [27]:
m=Mystem()

Now I will process the data using selected taggers, keeping both original and converted tags.

In [200]:
import nltk
pym_tags=[]
pym_original=[]
myst_tags=[]
myst_original=[]
nltk_tags=[]
nltk_original=[]
for i in text:
    if i in string.punctuation+'«»':
        pym_tags.append((i, 'PUNCT'))
        pym_original.append((i, 'PUNCT'))
        myst_tags.append((i, 'PUNCT'))
        myst_original.append((i, 'PUNCT'))
        nltk_tags.append((i, 'PUNCT'))
        nltk_original.append((i, 'PUNCT'))
    elif i.isdigit():
        pym_tags.append((i, 'NUM'))
        pym_original.append((i, 'NUM'))
        myst_tags.append((i, 'NUM'))
        myst_original.append((i, 'NUM'))
        nltk_tags.append((i, 'NUM'))
        nltk_original.append((i, 'NUM'))
    else:
        pym_tag=str(morph.parse(i)[0].tag).split(',')[0].split()[0]

        pym_original.append((i, pym_tag))
        try:
            myst_tag=m.analyze(i)[0]['analysis'][0]['gr'].split(',')[0]
        except:
            myst_tag='X'
        if '=' in myst_tag:
            myst_tag=myst_tag[:myst_tag.index('=')]
        myst_original.append((i, myst_tag))
        nltk_tag=nltk.pos_tag([i])[0][1]
        nltk_original.append((i, nltk_tag))
        if pym_tag in pym.keys():
            pym_tags.append((i, pym[pym_tag]))
        if pym_tag not in pym.keys():
            pym_tags.append((i, pym_tag))
        if myst_tag in myst.keys():
            myst_tags.append((i, myst[myst_tag]))
        if myst_tag not in myst.keys():
            myst_tags.append((i, myst_tag))
        if map_tag('en-ptb', 'universal', nltk_tag) in nlt.keys():
            nltk_tags.append((i, nlt[map_tag('en-ptb', 'universal', nltk_tag)]))
        if map_tag('en-ptb', 'universal', nltk_tag) not in nlt.keys():
            nltk_tags.append((i, map_tag('en-ptb', 'universal', nltk_tag)))    

# Evaluation

Let's see how the tagging went. I will list all the errors made by a tagger, and look at the tag from the dataset, the converted tag and the original tag of each incorrectly tagged word.

In [123]:
def compare(words, tag_list, tag_original):
    errors=[]
    for i in range(len(tag_list)):
        if tag_list[i] not in words:
            errors.append((words[i], tag_list[i], tag_original[i]))
    return errors

Mystem tagged 87 out of 389 words incorrectly. Mostly it didn't recognize pronouns, tagging them as just nouns instead. There were times, however, when Mystem tagged the words that were supposed to be unknown (see 'Ратлеф'), and was not at all wrong (giving 'Ратлеф' a noun tag). Also a lot of errors could be fixed by adding some context information, but I will cover that later.

In [126]:
len(compare(words, myst_tags, myst_original))

87

In [124]:
compare(words, myst_tags, myst_original)

[(('несколько', 'NUM'), ('несколько', 'ADV'), ('несколько', 'ADV')),
 (('на', 'ADP'), ('на', 'PREP'), ('на', 'PR')),
 (('Жильяр', 'PROPN'), ('Жильяр', 'NOUN'), ('Жильяр', 'S')),
 (('из', 'ADP'), ('из', 'PREP'), ('из', 'PR')),
 (('что', 'CONJ'), ('что', 'PRON'), ('что', 'SPRO')),
 (('больной', 'ADJ'), ('больной', 'NOUN'), ('больной', 'S')),
 (('сказано', 'ADJ'), ('сказано', 'VERB'), ('сказано', 'V')),
 (('что', 'CONJ'), ('что', 'PRON'), ('что', 'SPRO')),
 (('к', 'ADP'), ('к', 'NOUN'), ('к', 'S')),
 (('основано', 'ADJ'), ('основано', 'VERB'), ('основано', 'V')),
 (('на', 'ADP'), ('на', 'PREP'), ('на', 'PR')),
 (('Александра', 'PROPN'), ('Александра', 'NOUN'), ('Александра', 'S')),
 (('Теглева', 'X'), ('Теглева', 'ADJ'), ('Теглева', 'A')),
 (('Ольга', 'PROPN'), ('Ольга', 'NOUN'), ('Ольга', 'S')),
 (('в', 'ADP'), ('в', 'NOUN'), ('в', 'S')),
 (('в', 'ADP'), ('в', 'NOUN'), ('в', 'S')),
 (('Ратлеф', 'X'), ('Ратлеф', 'NOUN'), ('Ратлеф', 'S')),
 (('также', 'PART'), ('также', 'ADV'), ('также', '

pymorphy2 did better than Mystem, making a lot of similar errors, however. There is a visible disagreement between 'ADJ' and 'DET' tags, because pymorphy2 originally doesn't have the 'DET' tag, although it has various adjective tags. Almost all the errors are PROPN/NOUN, DET/ADJ(ADJF) and X/NOUN types, so there are almost no serious errors in this set.

In [201]:
len(compare(words, pym_tags, pym_original))

56

In [202]:
compare(words, pym_tags, pym_original)

[(('несколько', 'NUM'), ('несколько', 'ADV'), ('несколько', 'ADVB')),
 (('которые', 'DET'), ('которые', 'ADJ'), ('которые', 'ADJF')),
 (('Сам', 'DET'), ('Сам', 'ADJ'), ('Сам', 'ADJF')),
 (('Жильяр', 'PROPN'), ('Жильяр', 'NOUN'), ('Жильяр', 'NOUN')),
 (('сказано', 'ADJ'), ('сказано', 'VERB'), ('сказано', 'PRTS')),
 (('основано', 'ADJ'), ('основано', 'VERB'), ('основано', 'PRTS')),
 (('Александра', 'PROPN'), ('Александра', 'NOUN'), ('Александра', 'NOUN')),
 (('Теглева', 'X'), ('Теглева', 'ADJ'), ('Теглева', 'ADJS')),
 (('Ольга', 'PROPN'), ('Ольга', 'NOUN'), ('Ольга', 'NOUN')),
 (('Ратлеф', 'X'), ('Ратлеф', 'NOUN'), ('Ратлеф', 'NOUN')),
 (('также', 'PART'), ('также', 'CONJ'), ('также', 'CONJ')),
 (('запомнившиеся', 'ADJ'),
  ('запомнившиеся', 'VERB'),
  ('запомнившиеся', 'PRTF')),
 (('Мои', 'DET'), ('Мои', 'ADJ'), ('Мои', 'ADJF')),
 (('Андерсон', 'PROPN'), ('Андерсон', 'NOUN'), ('Андерсон', 'NOUN')),
 (('этим', 'DET'), ('этим', 'PRON'), ('этим', 'NPRO')),
 (('также', 'PART'), ('также', 'C

NLTK was not really meant to be for Russian, and it likes to think all Russian words are nouns, bringing stability to our evaluation.

In [129]:
len(compare(words, nltk_tags, nltk_original))

231

In [128]:
compare(words, nltk_tags, nltk_original)

[(('Опускались', 'VERB'), ('Опускались', 'NOUN'), ('Опускались', 'NN')),
 (('Я', 'PRON'), ('Я', 'NOUN'), ('Я', 'NN')),
 (('задал', 'VERB'), ('задал', 'NOUN'), ('задал', 'NN')),
 (('ей', 'PRON'), ('ей', 'NOUN'), ('ей', 'NN')),
 (('по-немецки', 'ADV'), ('по-немецки', 'NOUN'), ('по-немецки', 'NN')),
 (('несколько', 'NUM'), ('несколько', 'NOUN'), ('несколько', 'NN')),
 (('на', 'ADP'), ('на', 'NOUN'), ('на', 'NN')),
 (('которые', 'DET'), ('которые', 'NOUN'), ('которые', 'NN')),
 (('она', 'PRON'), ('она', 'NOUN'), ('она', 'NN')),
 (('отвечала', 'VERB'), ('отвечала', 'NOUN'), ('отвечала', 'NN')),
 (('невнятными', 'ADJ'), ('невнятными', 'NOUN'), ('невнятными', 'NN')),
 (('Сам', 'DET'), ('Сам', 'NOUN'), ('Сам', 'NN')),
 (('Жильяр', 'PROPN'), ('Жильяр', 'NOUN'), ('Жильяр', 'NN')),
 (('сделал', 'VERB'), ('сделал', 'NOUN'), ('сделал', 'NN')),
 (('из', 'ADP'), ('из', 'NOUN'), ('из', 'NN')),
 (('этого', 'PRON'), ('этого', 'NOUN'), ('этого', 'NN')),
 (('что', 'CONJ'), ('что', 'NOUN'), ('что', 'NN')),

Now let's try and give Mystem the original article as a text, so it has an opportunity to tag words according to the context.

In [206]:
import re
myst_context=[]
myst_context_orig=[]
for i in m.analyze(article):
    if 'analysis' in i.keys():
        tag=i['analysis'][0]['gr'].split(',')[0]
        tag_text=i['text']
        if '=' in tag:
            tag=tag[:tag.index('=')]
        myst_context_orig.append((tag_text, tag))
        if tag in myst.keys():
            myst_context.append((tag_text, myst[tag]))
        elif tag not in myst.keys():
            myst_context.append((tag_text, tag))
    elif 'analysis' not in i.keys() and i['text']==' ':
        pass
    elif 'analysis' not in i.keys() and re.findall('\.|,|!|:|;|\?|«|»|\(|\)+', i['text']):
        myst_context.append((re.findall('\.|,|!|:|;|\?|«|»|\(|\)+', i['text'])[0], 'PUNCT'))
        myst_context_orig.append((re.findall('\.|,|!|:|;|\?|«|»|\(|\)+', i['text'])[0], 'PUNCT'))
    else:
        myst_context.append((i['text'], 'X'))
        myst_context_orig.append((i['text'], 'X'))

However, as it can be seen, this didn't really help (the error rate for Mystem was originally 87).

In [209]:
len(compare(words, myst_context, myst_context_orig))

84

Making a conclusion, the best results were achieved by pymorphy2 (53 errors out of 389 words). Most of it's errord were more tag disagreement than actually incorrect tags. However, maybe it could have been different if I chose another text. Also NLTK POS-tagger should not be used for tagging Russian.